In [2]:
import numpy as np
import scipy as sp
from math import factorial,tanh
import itertools

import matplotlib.pyplot as plt
import matplotlib.path as mpath
import matplotlib.patches as mpatches
from matplotlib.collections import PatchCollection

import strawberryfields as sf
from strawberryfields.ops import*

colormap = np.array(plt.rcParams['axes.prop_cycle'].by_key()['color'])

Constructing the circuit

constants

In [4]:
r_squeezing = 0.5
cutoff = 7
theta1 = 0.5
theta2 = 1

U = np.array([[np.cos(theta1),-np.sin(theta1),0,0],
              [np.sin(theta1),np.cos(theta1),0,0],
              [0,0,np.cos(theta2),-np.sin(theta2)],
              [0,0,np.sin(theta2),np.cos(theta2)]])

In [5]:
prog = sf.Program(8)
with prog.context as q:
    S2gate(r_squeezing) | (q[0],q[4])
    S2gate(r_squeezing) | (q[1],q[5])
    S2gate(r_squeezing) | (q[2],q[6])
    S2gate(r_squeezing) | (q[3],q[7])

    Interferometer(U) | (q[4],q[5],q[6],q[7])
eng = sf.Engine('fock',backend_options={"cutoff_dim":cutoff})
state = eng.run(prog).state

/Users/gshilong/miniconda3/envs/strawberryfields/lib/python3.12/site-packages/strawberryfields/program.py:732: UserWarning: The circuit consists of 2 disconnected components.


In [7]:
probs = state.all_fock_probs()

In [8]:
probs = probs.reshape(*[cutoff]*8)
np.sum(probs)

0.9998583445961168

Analysis

Computation of the theoretical probability

In [9]:
from thewalrus import perm

In [10]:
def get_proba_output(U,input,output):
    # The two lines below are the extracted row and column indices.
    # For instance, for output=[3,2,1,0], we want list_rows=[0,0,0,1,1,2].
    # sum(.,[]) is a Python trick to flatten the list
    list_rows = sum([[i] * output[i] for i in range(len(output))],[])
    list_columns = sum([[i] * input[i] for i in range(len(input))],[])

    U_st = U[:,list_columns][list_rows,:]
    perm_squared = np.abs(perm(U_st,method="ryser"))**2
    denominator = np.prod([factorial(inp) for inp in input])*np.prod([factorial(out) for out in output])
    return perm_squared/denominator

def get_proba_input(input):
    chi = np.tanh(r_squeezing)
    n= np.sum(input)
    m = len(input)
    return (1 - chi**2)**m * chi**(2*n)

def get_proba(U,result):
    input,output = result[0:4],result[4:8]
    return get_proba_output(U,input,output) * get_proba_input(input)


Comparison between theory and simulation

In [13]:
print("theory: \t",get_proba(U,[0,0,0,0,0,0,0,0]))
print("simulation: \t",probs[0,0,0,0,0,0,0,0])
print("theory: \t",get_proba(U,[1,0,0,0,1,0,0,0]))
print("simulation: \t",probs[1,0,0,0,1,0,0,0])
print("theory: \t",get_proba(U,[1,0,0,0,0,1,0,0]))
print("simulation: \t",probs[1,0,0,0,0,1,0,0])

theory: 	 0.3825422953821255
simulation: 	 0.3825422953821258
theory: 	 0.06291578440230364
simulation: 	 0.06291578440230369
theory: 	 0.018776990012967093
simulation: 	 0.018776990012967107


Visualization

Make the probabilities sum to 1

In [15]:
probs[0,0,0,0,0,0,0,0] += 1 - np.sum(probs)
np.sum(probs)

1.0

sample

In [18]:
list_choices = list(itertools.product(*[range(cutoff)]*8))
list_choices[0]

(0, 0, 0, 0, 0, 0, 0, 0)

In [19]:
list_probs = [probs[list_choices[i]] for i in range(len(list_choices))]
list_probs[0]

0.38268395078600903

In [20]:
choice = list_choices[np.random.choice(range(len(list_choices)),p=list_probs)]
choice

(0, 4, 0, 0, 0, 4, 0, 0)